<a href="https://colab.research.google.com/github/MAT919/Coursera_Capstone/blob/master/Four_square_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up the project

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed
Libraries imported.


credentials of four square api

In [2]:
Client_id = 'XLUVMDU2R3LJPS5USBXBHCCKWAXWLWDGLCUKZCWUJTYS5YJD'
Client_Secret = 'UL5X2N4V21UZHSMDBTSFNFIYAGCQJAVMAHUUM43WTH2ZKAOI'
Version = '20180604'
Limit = 30
print('Your credentails:')
print('CLIENT_ID: ' + Client_id)
print('CLIENT_SECRET:' + Client_Secret)

Your credentails:
CLIENT_ID: XLUVMDU2R3LJPS5USBXBHCCKWAXWLWDGLCUKZCWUJTYS5YJD
CLIENT_SECRET:UL5X2N4V21UZHSMDBTSFNFIYAGCQJAVMAHUUM43WTH2ZKAOI


Convert the location of the hotel to it's longitude and latitude

In [3]:
address = '585 8th Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7553012 -73.99180667884616


Now ,search for specific place

In [4]:
search_query =  'Cafe'
radius = 500
print(search_query +'....OK')

Cafe....OK


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(Client_id, Client_Secret, latitude, longitude, Version, search_query, radius, Limit)
url

'https://api.foursquare.com/v2/venues/search?client_id=XLUVMDU2R3LJPS5USBXBHCCKWAXWLWDGLCUKZCWUJTYS5YJD&client_secret=UL5X2N4V21UZHSMDBTSFNFIYAGCQJAVMAHUUM43WTH2ZKAOI&ll=40.7553012,-73.99180667884616&v=20180604&query=Cafe&radius=500&limit=30'

In [6]:
results  =  requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4a757411c930671aa53322'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '56aa371be4b08b9a8d573538',
      'name': 'Theme Restaurant',
      'pluralName': 'Theme Restaurants',
      'primary': True,
      'shortName': 'Theme Restaurant'}],
    'delivery': {'id': '1944559',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/hard-rock-cafe-1501-broadway-new-york/1944559?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=1944559'},
    'hasPerk': False,
    'id': '428a8580f964a52083231fe3',
    'location': {'address': '1501 Broadway',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',

In [7]:
venues =  results['response']['venues']

In [8]:
DataFrame =  json_normalize(venues)
DataFrame.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,428a8580f964a52083231fe3,Hard Rock Cafe,"[{'id': '56aa371be4b08b9a8d573538', 'name': 'T...",v-1598715464,False,1501 Broadway,at W 43rd St,40.757035,-73.986611,"[{'label': 'display', 'lat': 40.75703518267677...",478,10036,US,New York,NY,United States,"[1501 Broadway (at W 43rd St), New York, NY 10...",1944559,https://www.seamless.com/menu/hard-rock-cafe-1...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN
1,4a6b805ff964a520d1ce1fe3,Europa Cafe,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1598715464,False,345 W 42nd St,btw 8th and 9th Ave,40.758202,-73.991562,"[{'label': 'display', 'lat': 40.75820159912109...",323,10036,US,New York,NY,United States,"[345 W 42nd St (btw 8th and 9th Ave), New York...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4e3041c362e1fbac611379c2,EZ Cafe,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1598715464,False,572 8th Ave,btwn 38th St. & 39th St.,40.754850,-73.991599,"[{'label': 'display', 'lat': 40.75485037344214...",53,10018,US,New York,NY,United States,"[572 8th Ave (btwn 38th St. & 39th St.), New Y...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b8f470df964a520205033e3,Europan Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1598715464,False,672 8th Ave,at W 43rd St.,40.757729,-73.989411,"[{'label': 'display', 'lat': 40.75772900046835...",337,10036,US,New York,NY,United States,"[672 8th Ave (at W 43rd St.), New York, NY 100...",71846,https://www.seamless.com/menu/europan-cafe-672...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,71340844,NaN
4,50ba5b67e4b054161c5f2271,cafe Mofongo,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S...",v-1598715464,False,316 W 39th St,Between eighth and ninth,40.755836,-73.992059,"[{'label': 'display', 'lat': 40.75583566192281...",63,10018,US,New York,NY,United States,"[316 W 39th St (Between eighth and ninth), New...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
Filtered = DataFrame[['id','name','location.lat','location.lng',
                      'categories']]
 
Filtered.head()

,id,name,location.lat,location.lng,categories
0,428a8580f964a52083231fe3,Hard Rock Cafe,40.757035,-73.986611,"[{'id': '56aa371be4b08b9a8d573538', 'name': 'T..."
1,4a6b805ff964a520d1ce1fe3,Europa Cafe,40.758202,-73.991562,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A..."
2,4e3041c362e1fbac611379c2,EZ Cafe,40.754850,-73.991599,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F..."
3,4b8f470df964a520205033e3,Europan Cafe,40.757729,-73.989411,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C..."
4,50ba5b67e4b054161c5f2271,cafe Mofongo,40.755836,-73.992059,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S..."


In [10]:
map  = folium.Map(location=[latitude, longitude],zoom_start=17)

folium.CircleMarker(
    [latitude, longitude],
    radius = 10,
    color = 'red',
    popup = 'Holiday inn',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6    
).add_to(map)

for lat, lng, label in zip(Filtered['location.lat'], Filtered['location.lng'],Filtered['name']):
  folium.Marker(
    [lat, lng],
    popup = label
).add_to(map)
map

Explore the venue

In [11]:
Venue_id = '4b8f470df964a520205033e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(Venue_id, Client_id, Client_Secret, Version)
url

'https://api.foursquare.com/v2/venues/4b8f470df964a520205033e3?client_id=XLUVMDU2R3LJPS5USBXBHCCKWAXWLWDGLCUKZCWUJTYS5YJD&client_secret=UL5X2N4V21UZHSMDBTSFNFIYAGCQJAVMAHUUM43WTH2ZKAOI&v=20180604'

In [12]:
results = requests.get(url).json()
results

results['response']['venue']['rating']

6.4

In [13]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(Venue_id, Client_id, Client_Secret, Version, limit)

results = requests.get(url).json()
results 

{'meta': {'code': 200, 'requestId': '5f4a75e17f58eb2f2f096b30'},
 'response': {'tips': {'count': 46,
   'items': [{'agreeCount': 3,
     'canonicalUrl': 'https://foursquare.com/item/5679da67498e912144aba41e',
     'createdAt': 1450826343,
     'disagreeCount': 0,
     'id': '5679da67498e912144aba41e',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'photo': {'createdAt': 1450826474,
      'height': 2340,
      'id': '5679daea498e845c9ede9872',
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'source': {'name': 'Foursquare for Android',
       'url': 'https://foursquare.com/download/#/android'},
      'suffix': '/108794774_Bg9OVXV7aIIl3AG_4CT6eCaHzwfehyslhGPmgTC1rJk.jpg',
      'visibility': 'public',
      'width': 4160},
     'photourl': 'https://fastly.4sqi.net/img/general/original/108794774_Bg9OVXV7aIIl3AG_4CT6eCaHzwfehyslhGPmgTC1rJk.jpg',
     'text': 'Casual and nice. Great to drop in here to kill an hour before a Broadway show.

In [14]:
tips = results['response']['tips']['items']
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) 
tips_df = tips_df[['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']]
tips_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,Casual and nice. Great to drop in here to kill an hour before a Broadway show. I tried Tiramisu and Red Velvet Slice. Both excellent and well priced. $7.55 inc a small coffee! Love it.,3,0,5679da67498e912144aba41e,Simon,A,108794774


Explore the location

In [37]:
address = '57 E 57th St, New York, NY 10022, United States'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.76230085 -73.97125668962013


In [38]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(Client_id, Client_Secret, latitude, longitude, Version, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XLUVMDU2R3LJPS5USBXBHCCKWAXWLWDGLCUKZCWUJTYS5YJD&client_secret=UL5X2N4V21UZHSMDBTSFNFIYAGCQJAVMAHUUM43WTH2ZKAOI&ll=40.76230085,-73.97125668962013&v=20180604&radius=500&limit=15'

In [39]:
results =  requests.get(url).json()
results = results['response']['groups'][0]['items']
results 

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-4a9eeabff964a520c83b20e3-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d1ed941735',
     'name': 'Spa',
     'pluralName': 'Spas',
     'primary': True,
     'shortName': 'Spa'}],
   'id': '4a9eeabff964a520c83b20e3',
   'location': {'address': '505 Park Avenue',
    'cc': 'US',
    'city': 'New York',
    'country': 'United States',
    'crossStreet': 'at 59th St',
    'distance': 155,
    'formattedAddress': ['505 Park Avenue (at 59th St)',
     'New York, NY 10022',
     'United States'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.763066029564804,
      'lng': -73.96971217558523}],
    'lat': 40.763066029564804,
    'lng': -73.96971217558523,
    'postalCode': '10022',
    'state': 'NY'},
   'n

In [40]:
Venue = json_normalize(results)
Venue.head(1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name
0,e-0-4a9eeabff964a520c83b20e3-0,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",4a9eeabff964a520c83b20e3,Mizu Salon,505 Park Avenue,at 59th St,40.763066,-73.969712,"[{'label': 'display', 'lat': 40.763066029564804, 'lng': -73.96971217558523}]",155,10022,US,New York,NY,United States,"[505 Park Avenue (at 59th St), New York, NY 10022, United States]","[{'id': '4bf58dd8d48988d1ed941735', 'name': 'Spa', 'pluralName': 'Spas', 'shortName': 'Spa', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_', 'suffix': '.png'}, 'primary': True}]",0,[],519406496,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
Venue =  Venue[['venue.id','venue.name','venue.location.lat','venue.location.lng']]
Venue.head(1)

,venue.id,venue.name,venue.location.lat,venue.location.lng
0,4a9eeabff964a520c83b20e3,Mizu Salon,40.763066,-73.969712


In [42]:
Venue_map = folium.Map(location=[latitude,longitude],zoom_start=15)

folium.CircleMarker(
    [latitude, longitude],
    radius = 10,
    popup = 'current location',
    fill = True,
    color = 'red',
    fill_color = 'red',
    fill_opacity = 0.5
).add_to(Venue_map)



for lat, lon, label in zip(Venue['venue.location.lat'],Venue['venue.location.lng'],Venue['venue.name']):
  folium.Marker(
      location = [lat,lon],
      popup = label,
  ).add_to(Venue_map)

Venue_map

Explore trending places

In [43]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(Client_id, Client_Secret, latitude, longitude, Version)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4a7eb504da622edb4a31ba'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d164941735',
      'name': 'Plaza',
      'pluralName': 'Plazas',
      'primary': True,
      'shortName': 'Plaza'}],
    'delivery': {'id': '2198019',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/ben--jerrys-times-square-200-w-44th-st-new-york/2198019?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=2198019'},
    'id': '49b7ed6df964a52030531fe3',
    'location': {'address': 'Broadway & 7th Ave',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'btwn W 42n

In [51]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
else:
  trending_venues = results['response']['venues']
  trending_venues_df = json_normalize(trending_venues)


trending_venues_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,location.address,location.crossStreet,location.lat,location.lng,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,49b7ed6df964a52030531fe3,Times Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'Plaza', 'pluralName': 'Plazas', 'shortName': 'Plaza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_', 'suffix': '.png'}, 'primary': True}]",Broadway & 7th Ave,btwn W 42nd & 47th St,40.756439,-73.986621,1450,10036,US,Theater District,New York,NY,United States,"[Broadway & 7th Ave (btwn W 42nd & 47th St), New York, NY 10036, United States]",2198019,https://www.seamless.com/menu/ben--jerrys-times-square-200-w-44th-st-new-york/2198019?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=2198019,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png


In [53]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
else:
  trending_map = folium.Map([latitude,longitude],zoom_start=15)
  folium.CircleMarker(
    [latitude, longitude],
    radius = 10,
    popup = 'current location',
    fill = True,
    color = 'red',
    fill_color = 'red',
    fill_opacity = 0.5
).add_to(trending_map)
  for lat, lon, label in zip(trending_venues_df['location.lat'],trending_venues_df['location.lng'],trending_venues_df['name']):
    folium.Marker(
        location = [lat,lon],
        popup = label
    ).add_to(trending_map)

trending_map


 